In [ ]:
pip install google_play_scraper

In [ ]:
import os

# verifying and removing the file, if exists from google drive

if os.path.exists("app_review.csv"):
  os.remove("app_review.csv")
  print("The file is deleted.")
else:
  print("The file does not exist.")

The file does not exist.


In [ ]:
import multiprocessing
from google_play_scraper import Sort, reviews
import re
import sys


for i in range (1,4):
  try:
    result, continuation_token = reviews(
      'com.google.android.apps.youtube.kids',
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.RATING, # None, defaults to Sort.MOST_RELEVANT
      count=i, # defaults to 100
      filter_score_with=5
    )

    # If we pass 'continuation_token' as an argument to the reviews function at this point,
    # it will crawl the items after "i-th" review item.
          
    result, _ = reviews(
      'com.google.android.apps.youtube.kids',
      continuation_token=continuation_token # defaults to None(load from the beginning)
    )

    # remove unicode characters from data
    txt_extract = str(result)
    encoded_string = txt_extract.encode("ascii", "ignore")
    decode_string = encoded_string.decode()
    txt = decode_string
        
    # pre-cleaning of each review before storing into the file app_review.csv
    txt = re.sub("'","",txt)
    txt = re.sub("\"","",txt)
    txt = re.sub("#","",txt)
        
    # extracting username
    user = txt.split('userName: ')[1].split(', userImage')[0]
        
    # extracting rating
    rating = txt.split('score: ')[1].split(', thumbsUpCount')[0]
        
    # extracting likes
    upvote = txt.split('thumbsUpCount: ')[1].split(', reviewCreatedVersion')[0]
        
    # breaking the datetime in each review to year, month, day, hour, minute, second
    datetime = re.search('datetime.datetime\((.+?)\),', txt).group(1)
    datetime = re.sub(", ",",",datetime)
        
    # extracting review
    review = re.search('content: (.+?), score', txt).group(1)
    review = re.sub("!","",review)
    review = re.sub(",","",review)
    review = "'"+review+"'"

    print("i : ",i)
    print(user)
    print(datetime)
    print(rating)
    print(upvote)
    print(review)

    # created and opened file in append mode
    file = open('app_review.csv','a')
    file.write(user+', '+rating+', '+upvote+', '+review+', '+datetime+'\n')

  finally:
    file.close()


i :  1
Yusuf Patel
2019,1,29,10,5
5
0
'nice'
i :  2
Vishakha Muley
2019,11,11,10,56,55
5
0
'Very nice app'
i :  3
Nathan Agda
2017,1,5,0,2,36
5
0
'Love it'


In [ ]:
# downloading the .csv file into local drive

from google.colab import files
files.download('app_review.csv')